In [5]:
# Program 1: generate original and curated source file.
def access(oname):
    from access_token import AccessToken
    t = AccessToken('agroportal.config')
    from sem_art import Semart 
    s = Semart(t, oname)
    from py4j.java_gateway import JavaGateway
    gateway = JavaGateway()
    # read the original ontology file
    gateway.loadOntology(s.get_source())
    # save the original ontology
    st = gateway.saveOntology("C:/Users/arkop/git/meta-fair/metafair/src/test/resources/"+oname+"-orig.owl")
    print(st)
    # remove all original metadata annotations
    gateway.deleteAnnotations()
    # get all metadata and annotate
    for m in s.get_metadata():
        ms = s.get_metadata_value(m)
        # ignore if no value is curated for the metadata
        if(ms[1]==None): 
            continue
        # for 'contacts' only pick up the name    
        if(m=="contacts"):
            if(len(ms[1])==0):
                continue
            for mi in ms[1]:
                print(str(ms[0]))
                print(str(mi['name']))
                gateway.addAnnotations(str(ms[0]), str(mi['name']))
            continue
        # for tuple or list metadata value assert annotations separately for each member
        if type(ms[1]) is list or type(ms[1]) is tuple:
            if(len(ms[1])==0):
                continue
            for mi in ms[1]:
                gateway.addAnnotations(str(ms[0]), str(mi))
            continue
        # general singleton metadata 
        gateway.addAnnotations(str(ms[0]), str(ms[1]))
    # save the curated ontology
    st = gateway.saveOntology("C:/Users/arkop/git/meta-fair/metafair/src/test/resources/" + oname +"-curated.owl")
    print(st)
ms = access("AGRO")


loading all metadata from http://data.agroportal.lirmm.fr//ontologies/AGRO/latest_submission?display=all&apikey=e4b3eac1-55d2-4df7-bd9d-cd4e28a5c43d
loading all metadata from http://data.agroportal.lirmm.fr//ontologies/AGRO/download?apikey=e4b3eac1-55d2-4df7-bd9d-cd4e28a5c43d
Successfully saved ontology!
Successfully saved ontology!


In [4]:
with open('.config') as f:
    line = f.readline()
    tokens = line.split('=')
    key = tokens[1].strip()
    print(key)

4665c07b-ff68-4764-9b3c-b465c17c21ca


In [ ]:
import json
import requests

headers = {
            'accept': 'application/json;charset=UTF-8',
            'Content-Type': 'application/json;charset=UTF-8',
        }

json_data = {
            'ontologyUri': 'https://raw.githubusercontent.com/arsarkar/meta-fair/main/metafair/src/test/resources/AGRO-orig.owl',
        }

response = requests.post('http://localhost:1099/assessOntology', headers=headers, json=json_data)

print(response.text)

In [1]:
def faireval()->float:
    from foops import Foops
    foop = Foops()
    foop.check('https://raw.githubusercontent.com/arsarkar/meta-fair/main/metafair/src/test/resources/AGRO-orig.owl')
    return foop.get_overall_score()
score = faireval()
print(score)

0.48611107


In [4]:
def faireval()->str:
    from ofaire import OFaire
    from access_token import AccessToken
    t = AccessToken('industryportal.config')
    of = OFaire('industryportal',t)
    of.check('IOF-CORE')
    return of.error
    # return of.get_overall_score()
score = faireval()
print(score)

Not Found


In [20]:
from access_token import AccessToken
t = AccessToken('industryportal.config')
import requests
import json
from requests.exceptions import HTTPError
uri = 'IOF-CORE'
try:
    response = requests.get(t.url+"fair", params={'portal':'industryportal', 'ontologies':uri, 'apikey':t.auth})
    if response.status_code == 200:
        print(json.dumps(response.json()))
    else:
        print(response.text)
except HTTPError as e:
    print(e.response.reason)

{"ontologies": {"IOF-CORE": {"Findable": {"F1": {"label": "Ontologies and ontology metadata are assigned a globally unique and persistent identifier. ", "results": {"F1Q1": {"question": "Does the ontology have a \"local\" identifier, i.e., a globally unique and potentially permanent identifier assigned by the developer (or developing organization)?", "score": 9.0, "explanation": "Ontology URI is present and valid.", "properties": {"owl:ontologyIRI": "https://purl.industrialontologies.org/ontology/core/Core"}, "maxCredits": 9.0, "points": [{"explanation": "Ontology URI is absent.", "score": 0.0}, {"explanation": "Ontology URI is present but invalid.", "score": 3.0}, {"explanation": "Ontology URI is present and valid.", "score": 9.0}]}, "F1Q2": {"question": "Does the ontology provide an additional \"external\" identifier, i.e., a guarantee globally unique and persistent identifier assigned by an accredited body? If yes, is the external identifier a DOI?", "score": 0.0, "explanation": "Ex

In [1]:
import requests
import json
from requests.exceptions import HTTPError
try:
    response = requests.get('https://fair-checker.france-bioinformatique.fr/api/check/metrics_all', 
                            params={'url':'https://raw.githubusercontent.com/arsarkar/meta-fair/main/metafair/src/test/resources/AGRO-curated.owl'})
    if response.status_code == 200:
        print(json.dumps(response.json()))
        result = response.json()
    else:
        print(response.text)
except HTTPError as e:
    print(e.response.reason)
type(result)
s = 0
for x in result:
    s = s + int(x['score']) 
print(s)

[{"metric": "F1A", "score": "2", "target_uri": "https://raw.githubusercontent.com/arsarkar/meta-fair/main/metafair/src/test/resources/AGRO-curated.owl", "eval_time": "0:00:00.000077", "recommendation": "No recommendation, metric validated", "comment": "INFO - Evaluating metrics Unique IDs\nINFO - Checking if the URL is reachable, status code: 200\nINFO - Status code is OK, meaning the url is Unique.\n"}, {"metric": "F1B", "score": "1", "target_uri": "https://raw.githubusercontent.com/arsarkar/meta-fair/main/metafair/src/test/resources/AGRO-curated.owl", "eval_time": "0:00:00.096313", "recommendation": "No recommendation, metric validated", "comment": "INFO - Evaluating metrics Persistent IDs\nINFO - Strong evaluation:\nINFO - Checking if there is either schema:identifier or dct:identifier property in metadata\nINFO - None of those property were found in metadata\nINFO - Trying weaker evaluation\nINFO - Weak evaluation:\nINFO - Checking that at least one namespace from identifiers.org i